In [1]:
import numpy as np
DTYPE = np.float64

import matplotlib.pyplot as plt
from tqdm import tqdm


In [2]:
# Dimensions of the parameter space:
D = 2

# Mesh Size
N = 100

In [28]:
def floryHuggins_2components(phi:DTYPE, chi:DTYPE):
    f = phi*np.log(phi) + (1-phi)*np.log(1-phi) + chi*phi*(1-phi)
    return f

In [104]:
class c2c2:
    def __init__(self, beta, phi_global, chi, D, N):
        self.beta = beta
        self.phi_global = phi_global
        self.D = D
        self.chi = chi
        self.N = N
        
    def createMesh(self):
        mesh = np.zeros((self.D, self.N), dtype = DTYPE)
        
        for _ in range(self.D):
            for idx in range(self.N):
                mesh[_][idx] = (idx+1)/(self.N+1)

        return mesh

    def metropolisStep(self, current_idx_phi_11, current_idx_eta_1):
        mesh = self.createMesh()
        
        current_phi_12 = (self.phi_global - mesh[1][current_idx_eta_1]*mesh[0][current_idx_phi_11])/(1-mesh[1][current_idx_eta_1])

        proposal_idx_phi_11 = 0
        proposal_idx_eta_11 = 0
        
        if current_idx_phi_11 == 0:
            proposal_idx_phi_11 = current_idx_phi_11 + 1
        
        elif current_idx_eta_1 == 0:
            proposal_idx_eta_1 = current_idx_eta_1 + 1
        
        elif current_idx_phi_11 == self.N-1:
            proposal_idx_phi_11 = current_idx_phi_11 - 1
            # proposal_idx_eta_1 = current_idx_eta_1
        
        elif current_idx_eta_1 == self.N-1:
            proposal_idx_eta_1 = current_idx_eta_1 - 1
        
        else:
            p = np.random.uniform(0, 1)
            
            if p <= 0.25:
                proposal_idx_phi_11 = current_idx_phi_11 + 1
                # proposal_idx_eta_1 = current_idx_eta_1
            elif 0.25 <= p  and p <= 0.5:
                proposal_idx_phi_11 = current_idx_phi_11 - 1
                # proposal_idx_eta_1 = current_idx_eta_1
            elif 0.5 <= p  and p <= 0.75:
                # proposal_idx_phi_11 = current_idx_phi_11
                proposal_idx_eta_1 = current_idx_eta_1 + 1
            else:
                # proposal_idx_phi_11 = current_idx_phi_11 + 1
                proposal_idx_eta_1 = current_idx_eta_1 - 1

        proposal_phi_12 = (self.phi_global - mesh[1][proposal_idx_eta_1]*mesh[0][proposal_idx_phi_11])/(1-mesh[1][proposal_idx_eta_1])
        
        if proposal_phi_12 <= 1 and proposal_phi_12 >= 0:
            f_1_current = floryHuggins_2components(mesh[0][current_idx_phi_11], self.chi) 
            f_2_current = floryHuggins_2components(current_phi_12, self.chi) 

            f_current = mesh[1][current_idx_eta_1]*f_1_current  + (1-mesh[1][current_idx_eta_1])*f_2_current


            f_1_proposal = floryHuggins_2components(mesh[0][proposal_idx_phi_11], self.chi)
            f_2_proposal = floryHuggins_2components(proposal_phi_12, self.chi)
            
            f_proposal = mesh[1][proposal_idx_eta_1]*f_1_proposal + mesh[1][proposal_idx_eta_1]*f_2_proposal

            delta_f = f_proposal - f_current

            if delta_f < 0 or np.random.uniform(0, 1) < np.exp(-self.beta*delta_f):
                return [proposal_idx_phi_11, proposal_idx_eta_1]
            else:
                return [current_idx_phi_11, current_idx_eta_1]
        else:
            return [current_idx_phi_11, current_idx_eta_1]

    def simulate(self, start_idx_phi_11, start_idx_eta_1, nSteps):
        walk = []
        walk.append([start_idx_phi_11, start_idx_eta_1])
            
        for i in range(nSteps):
            [new_idx_phi_11, new_idx_eta_1] = self.metropolisStep(start_idx_phi_11, start_idx_eta_1)
            walk.append([new_idx_phi_11, new_idx_eta_1])

            start_idx_phi_11, start_idx_eta_1 = new_idx_phi_11, new_idx_eta_1
            # print([new_phi_11, new_eta_1])
        
        return walk

In [105]:
m = c2c2(beta = 1, phi_global = 0.223, chi = 2.778, D = 2, N = 100)

In [106]:
m.simulate(N-1, N-1, 50)

UnboundLocalError: cannot access local variable 'proposal_idx_eta_1' where it is not associated with a value

In [57]:
mesh = np.zeros((D, N))

In [58]:
np.shape(mesh)

(2, 100)

In [59]:
mesh = np.zeros((D, N), dtype = DTYPE)
for _ in range(D):
    for idx in range(N):
        mesh[_][idx] = (idx+1)/(N+1)

In [60]:
mesh

array([[0.00990099, 0.01980198, 0.02970297, 0.03960396, 0.04950495,
        0.05940594, 0.06930693, 0.07920792, 0.08910891, 0.0990099 ,
        0.10891089, 0.11881188, 0.12871287, 0.13861386, 0.14851485,
        0.15841584, 0.16831683, 0.17821782, 0.18811881, 0.1980198 ,
        0.20792079, 0.21782178, 0.22772277, 0.23762376, 0.24752475,
        0.25742574, 0.26732673, 0.27722772, 0.28712871, 0.2970297 ,
        0.30693069, 0.31683168, 0.32673267, 0.33663366, 0.34653465,
        0.35643564, 0.36633663, 0.37623762, 0.38613861, 0.3960396 ,
        0.40594059, 0.41584158, 0.42574257, 0.43564356, 0.44554455,
        0.45544554, 0.46534653, 0.47524752, 0.48514851, 0.4950495 ,
        0.5049505 , 0.51485149, 0.52475248, 0.53465347, 0.54455446,
        0.55445545, 0.56435644, 0.57425743, 0.58415842, 0.59405941,
        0.6039604 , 0.61386139, 0.62376238, 0.63366337, 0.64356436,
        0.65346535, 0.66336634, 0.67326733, 0.68316832, 0.69306931,
        0.7029703 , 0.71287129, 0.72277228, 0.73